In [ ]:
# %config InlineBackend.figure_format = 'retina'
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import itertools
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

sns.set()

# Homework 4

Austin Gill

* 5: 2, 10abd
* 9: 1, 2
* 10: 1, 2
* Text addition

## Problem 5.2

Using Veranda and the basic motion algorithm, place a set of obstacles that cause the robot to cycle and not find the goal. In other words, build a robot trap.

## Problem 5.10.a

Implement Bug 1 in Veranda. Screen capture the results.

## Problem 5.10.b

Implement Bug 2 in Veranda. Screen capture the results.

## Problem 5.10.d

Implement Tangent Bug in Veranda. Screen capture the results.

## Problem 9.1

Assume that you are working in a large event center which has beacons located around the facility. Estimate the location of a robot, $(a,b,c)$, if the $(x,y,z) $ location of the beacon and the distance from the beacon to the robot, $d$, are given in the table below.

| $x$ | $y$ | $z$ | $d$ |
|-----|-----|-----|-----|
| 884 | 554 | 713 | 222 |
| 120 | 703 | 771 | 843 |
| 938 | 871 | 583 | 436 |
| 967 | 653 | 46  | 529 |
| 593 | 186 | 989 | 610 |

First, we plot the spheres to get a good idea of what we're looking at.

In [ ]:
centers = [
    (884, 554, 713),
    (120, 703, 771),
    (938, 871, 583),
    (967, 653, 46),
    (593, 186, 989),
]

radii = [222, 843, 436, 529, 610]

spheres = list(zip(centers, radii))

In [ ]:
u = np.linspace(0, 2 * np.pi, 20)
v = np.linspace(0, np.pi, 20)

circles = []

# Generate the points on the spheres
for center, r in spheres:
    x = center[0] + r * np.outer(np.cos(u), np.sin(v))
    y = center[1] + r * np.outer(np.sin(u), np.sin(v))
    z = center[2] + r * np.outer(np.ones(np.size(u)), np.cos(v))
    circles.append((x, y, z))

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect('equal')
ax.view_init(azim=-10, elev=15)

for x, y, z in circles:
    ax.plot_surface(x, y, z, linewidth=0.05, alpha=0.6)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

# TODO: <rant></rant>
# ax.set_xlim(0, 1000) and ax.set_xbound(0, 1000) set the *axis* bounds
# not the data bounds, so the spheres still get plotted outside the axis...
ax.set_xlim(0, 1000)
ax.set_ylim(0, 1000)
ax.set_zlim(0, 1000)

plt.show()

Now how to find the intersection point? For simplicity's sake, I will use the optimization technique given as the first example in the textbook. For this, we build an error function

$$f(x, y, z) = \sum_i \left( \sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2} - r_i \right)^2$$

and find the point $(x, y, z)$ that minimizes the error and claim that that point is as close to an intersection of the spheres as we're going to get.

I will modify this error function to also take in a vector $\vec s$ of sphere indices to use. This way I can loop through all combinations of 3, 4, and 5 of the given spheres.

In [ ]:
def E(v, s):
    """Error function for the given set of spheres."""
    x, y, z = v
    e = 0
    for sphere in (spheres[i] for i in s):
        (xi, yi, zi), ri = sphere
        e += (np.sqrt((x - xi)**2 + (y - yi)**2 + (z - zi)**2) - ri)**2

    return e

In [ ]:
# A generator of all the sphere index vectors
ss = itertools.chain.from_iterable(itertools.combinations(range(5), i) for i in [3, 4, 5])

In [ ]:
results = []
for s in ss:
    results.append(sp.optimize.minimize(E, x0=(800, 400, 400), args=(s,)))

Note that the `minimize` function can fail, in which case the returned `OptimizeResult` object has the `success` field set to `False`.

In [ ]:
# Lists of (x, y, z) tuples
positions = np.array([r.x for r in results if r.success])

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Unzip the list of tuples into a tuple of lists
ax.scatter(*zip(*positions), alpha=0.7)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

plt.title('Optimization results for all combinations of 3, 4, and 5 spheres')

plt.show()

This gives us a bunch of potential intersection points. Yay. So we remove the two outliers and plot the centroid.

In [ ]:
x, y, z = zip(*positions)

filtered = []

# Filter out the outliers
for xi, yi, zi in zip(x, y, z):
    if xi > 860:
        filtered.append((xi, yi, zi))

filtered = np.array(filtered)

centroid = np.mean(filtered, axis=0)
print(f'centroid: {centroid}')

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Unzip the list of tuples into a tuple of lists
ax.scatter(*zip(*filtered), alpha=0.7, label='Intersections')
ax.scatter(*centroid, label='Centroid')

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

plt.title('Optimization results with outliers removed')
plt.legend()

plt.show()

So the estimated robot position is near $(886.03, 442.21, 521.77)$. If matplotlib would respect the `ax.set_*lim()` function calls when I plot the sphere surfaces, I could generate a helpful plot showing the spheres and the estimated position. But I've spent far too much of my life going through the first ten pages of my Google searches (I even used Google over DuckDuckGo for this) for me to try any more.

## Problem 9.2

If you are using a laser diode to build a distance sensor, you need some method to determine the travel time. Instead of using pulses and a clock, try using phase shifts. What is the wavelength of the modulated frequency of 10MHz? If you measure a 10 degree phase shift, this value corresponds to what distances? What if the phase shift measurement has noise: zero mean with standard deviation 0.1? How does one get a good estimate of position if the ranges to be measured are from 20 meters to 250 meters?

## Problem 10.1

Assume you have a laser triangulation system as shown in Fig. 10.2 given by (10.1) and that $f=8$mm, $b=30$cm. What are the ranges for $\alpha$ and $u$ if we need to measure target distances in a region (in cm) $20<z<100$ and $10<x<30$?

$$x = \frac{b u}{f\cot \alpha + u},  \quad
  z = \frac{b f}{f\cot \alpha + u}$$
  
![Fig 10.2](figures/lasertriangulation2.svg)

## Problem 10.2

Assume you have two cameras that are calibrated into a stereo pair with a baseline of $10$cm, and focal depth of $7$mm. If the error is $10\%$ on $v_1$ and $v_2$, $v_1=2$mm and $v_2=3$mm, what is the error on the depth measurement $z$? Your answer should be a percentage relative to the error free number. Hint: If $v_1=2$ then a $10\%$ error ranges from $1.8$ to $2.2$. (Although not required, another way to approach this problem is the total differential from calculus.)